In [ ]:
include("../test/test_utils.jl")
using ExpandLGF
using OffsetArrays
using Printf
using ThreadPools

### Common Parameters

In [ ]:
split_stencils_3D = ExpandLGF.StandardDifference3D.([2, 4, 6, 8])
split_stencils_2D = ExpandLGF.StandardDifference2D.([2, 4, 6, 8])
left_mehrstellen_stencils = [ExpandLGF.LeftMehrstellen4(), ExpandLGF.LeftMehrstellen6()]
full_mehrstellen_stencils = [ExpandLGF.Mehrstellen4(), ExpandLGF.Mehrstellen6()]

split_stencil_names = ["LGF_2", "LGF_4", "LGF_6", "LGF_8"]
mehrstellen_names = ["MEH_4", "MEH_6"]

mkpath("../flups");

println("Calculating LGF values with $(Threads.nthreads()) threads.")

### 3D Data Files

In [ ]:
N = 32
near_terms = 12
far_terms = 8
rtol = 0
atol = 5e-16

for (stencil, name) in zip(split_stencils_3D, split_stencil_names)

    ELGF = ExpandLGF.EvaluateLGF(stencil, near_terms, far_terms, rtol, atol)
    lgf = OffsetArray(zeros(Float64, (N, N, N)), 0:N-1, 0:N-1, 0:N-1)
    evaluation_indices = sorted_indices_3D(N-1)

    println("Evaluating $(name) for indices [0, $(N-1)]^3 ... ")
    println("Total evaluations: $(length(evaluation_indices))")
    println("Using quadrature for |n| < $(ELGF.rad_)")

    @time ThreadPools.@qthreads for index in evaluation_indices
        lgf[index...] = ELGF(index)
    end

    # Build test array and evaluate the residual
    test = generate_full_test_block(stencil, lgf)
    residual = apply_stencil(stencil, test)
    handle_origin!(residual)
    max_residual, index = findmax(abs, residual)
    @printf("Max residual: %1.3e", max_residual)
    print(" at index $(Tuple(index))\n\n")

    # Build a full data array and write to file
    out = test[0:N-1, 0:N-1, 0:N-1]
    open("../flups/$(name)_3d_$(N).ker", "w") do file
        write(file, out)
    end
end

### 2D Data Files
In 2D the Far-Field expansion is defined up to constant, which can be determined in one of two ways:
 1. by matching the near-field and far-field expansion values at a point where both are sufficiently accurate
 2. by evaluating a singular integral with high precision

Here we choose the former strategy, which is handled automatically by the `ExpandLGF` structure. To translate this into C code, the tabulated data is offset by this constant, and the far-field expansion is implemented without an offset.

In [ ]:
N = 32
near_terms = 12
far_terms = 8
rtol = 0
atol = 1e-15

for (stencil, name) in zip(split_stencils_2D, split_stencil_names)

    ELGF = ExpandLGF.EvaluateLGF(stencil, near_terms, far_terms, rtol, atol)
    lgf = OffsetArray(zeros(Float64, (N, N)), 0:N-1, 0:N-1)
    evaluation_indices = sorted_indices_2D(N-1)

    println("Evaluating $(name) for indices [0, $(N-1)]^2 ... ")
    println("Total evaluations: $(length(evaluation_indices))")
    println("Using quadrature for |n| < $(ELGF.rad_)")

    @time ThreadPools.@qthreads for index in evaluation_indices
        lgf[index...] = ELGF(index)
    end

    # Build test array and evaluate the residual
    test = generate_full_test_block(stencil, lgf)
    residual = apply_stencil(stencil, test)
    handle_origin!(residual)
    max_residual, index = findmax(abs, residual)
    @printf("Max residual: %1.3e", max_residual)
    print(" at index $(Tuple(index))\n\n")

    out = test[0:N-1, 0:N-1]
    open("../flups/$(name)_2d_$(N).ker", "w") do file
        write(file, out)
    end
end

### Mehrstellen Data Files

The Mehrstellen stencils are not dimension-split, so near-field entries must be handled directly with quadrature. For efficiency this is done using a thread pool, but the code below may take up to two hours to run on four threads. 

In [ ]:
###
# Left Mehrstellen Stencils
###
N = 64
far_terms = 8
rtol = 0
atol = 1e-13
maxevals = 10^11

for (stencil, name) in zip(left_mehrstellen_stencils, mehrstellen_names)

    # Integrate to get indices i1 >= i2 >= i3
    ELGF = ExpandLGF.EvaluateLGF(stencil, far_terms, rtol, atol, maxevals)
    lgf = OffsetArray(zeros(Float64, (N, N, N)), 0:N-1, 0:N-1, 0:N-1)
    evaluation_indices = sorted_indices_3D(N-1)

    println("Total evaluations: $(length(evaluation_indices))")
    println("Evaluating $(name) for indices [0, $(N-1)]^3 ... ")
    println("Using quadrature for |n| < $(ELGF.rad_)")

    @time ThreadPools.@qthreads for index in evaluation_indices
        # println("Calculating index $(index) on thread $(Threads.threadid())")
        lgf[index...] = ELGF(index)
    end
    
    # Build test array and evaluate the residual
    test = generate_full_test_block(stencil, lgf)
    residual = apply_stencil(stencil, test)
    handle_origin!(residual, stencil)
    max_residual, index = findmax(abs, residual)
    @printf("Max residual: %1.3e", max_residual)
    print(" at index $(Tuple(index))\n\n")

    # Build a full data array and write to file
    out = test[0:N-1, 0:N-1, 0:N-1]
    open("../flups/$(name)_left_3d_$(N).ker", "w") do file
        write(file, out)
    end
end

In [ ]:
###
# Full Mehrstellen Stencils
###
N = 64
far_terms = 8
rtol = 0
atol = 1e-10
maxevals = 10^11

for (stencil, name) in zip(full_mehrstellen_stencils, mehrstellen_names)

    # Integrate to get indices i1 >= i2 >= i3
    ELGF = ExpandLGF.EvaluateLGF(stencil, far_terms, rtol, atol, maxevals)
    lgf = OffsetArray(zeros(Float64, (N, N, N)), 0:N-1, 0:N-1, 0:N-1)
    evaluation_indices = sorted_indices_3D(N-1)

    println("Evaluating $(name) for indices [0, $(N-1)]^3 ... ")
    println("Total evaluations: $(length(evaluation_indices))")
    println("Using quadrature for |n| < $(ELGF.rad_)")

    @time ThreadPools.@qthreads for index in evaluation_indices
        # println("Calculating index $(index) on thread $(Threads.threadid())")
        lgf[index...] = ELGF(index)
    end
    
    # Build test array and evaluate the residual
    test = generate_full_test_block(ExpandLGF.left_stencil(stencil), lgf)
    residual = apply_stencil(ExpandLGF.left_stencil(stencil), test)
    handle_origin!(residual, stencil)
    max_residual, index = findmax(abs, residual)
    @printf("Max residual: %1.3e", max_residual)
    print(" at index $(Tuple(index))\n\n")

    # Build a full data array and write to file
    out = test[0:N-1, 0:N-1, 0:N-1]
    open("../flups/$(name)_full_3d_$(N).ker", "w") do file
        write(file, out)
    end
end